In [1]:
# Código do grupo JP : Pedro Ribeiro 117367 e João Sousa 117368 MEB-IST 2025/26
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import plotly.graph_objects as go

#Parameters
Vo = 250          #L
Qp = 100   #L/h         
sigma_a = [0.9,0.3]
t_end = 10        #h
Cc_0 = [2,5.5]


def ode_fcn (t, y, Qp, Vo, sigma_a):

    Cc = y[0]                             
    dCc_dt = (-Qp / Vo) * (1 - sigma_a) * Cc # Diafiltration Formula

    return [dCc_dt]


def sol_ode(Cc_0,Vo, Qp, sigma_a, t_end): #This is just a function, no ODE's are being solved

    #function to solve
    fun = lambda t, y: ode_fcn(t, y, Qp, Vo, sigma_a) 
    
    y0 = [Cc_0] 

    # time span
    t_span = (0, t_end) # array with two numbers, 0 and t_end
    t_eval = np.linspace(*t_span, 1000) #Array with 100 elements to build the ODE's, step by step
    
    #Solve the differential equation
    solution = solve_ivp(fun, t_span, y0, method='LSODA', t_eval=t_eval, rtol=1e-8, atol=1e-8) #tolerancias relativas 

    #Extract the solution
    t = solution.t
    Cc = solution.y[0]
    # nD = (Qp*t)/Vo
    nD = [(Qp*x)/Vo for x in t]  #matriz do valor do produto, neste caso a nD

    return t, Cc, nD 


fig1 = go.Figure()

# Lactase
t, Cc, nD = sol_ode(Cc_0[0], Vo, Qp, sigma_a[0], t_end)
fig1.add_trace(go.Scatter(x= nD , y= Cc, mode='lines', name=f'Diafiltration Process - Lactase'))

# Impureza
t, Cc, nD = sol_ode(Cc_0[1], Vo, Qp, sigma_a[1], t_end)
fig1.add_trace(go.Scatter(x= nD , y= Cc, mode='lines', name=f'Diafiltration Process - Impurity'))


fig1.update_layout(title="Diafiltration Process",
                   xaxis_title="nD - Diafiltration Number",
                   yaxis_title="Cc - Concentration in retentate",
                   width=800)
fig1.show()